In [101]:
import pandas as pd
import seaborn as sns
import scipy as stats
import requests, json, collections, time, urllib, csv
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

#make it so that we only show first 4 decimals for floats
np.set_printoptions(precision=4,suppress=True)

# visualization
%matplotlib inline
import seaborn as sns

In [2]:
# Global Variables
MIN_YEAR = 1976
MAX_YEAR = 2016

In [23]:
# Test Variables
TEST_YEAR = 2003
DRIVER_ADDRESS = '/Users/Jon/Documents/Post_Grad/Metis/nbaProject/chromeDriver'

In [39]:
ADVANCED_PARAMS = {
    'PlayerExperience' : 'Rookie',
    'SeasonType' : 'Regular Season',
    'Season' : -1
    }

# cycle through all years
for year in list(range(MIN_YEAR, MAX_YEAR)):
    year = str(year) + '-' + str(year + 1)[2:4]

In [21]:
base_url = 'http://stats.nba.com/players/advanced/#!?'

In [40]:
params = ADVANCED_PARAMS
params['Season'] = '2003-04'

params_string = urllib.parse.urlencode(params).replace("+", "%20")
params_string

'PlayerExperience=Rookie&SeasonType=Regular%20Season&Season=2003-04'

In [41]:
driver = webdriver.Chrome(DRIVER_ADDRESS)
driver.get(base_url + params_string)

In [99]:
soup = BeautifulSoup(driver.find_element_by_class_name('stats-container').text, 'lxml')

In [76]:
r = requests.get(base_url + params_string)

In [100]:
soup

<html><body><p>Stats Home
Players
Teams
Advanced
Scores
Schedule
SEARCH FOR A PLAYER OR TEAM
Stats Home / Players / Advanced
Players
General
Advanced
SEASON
2016-17
2015-16
2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97
SEASON TYPE
Preseason
Regular Season
Playoffs
All-Star
SEASON SEGMENT
All Games
Last Game
Last 2 Games
Last 3 Games
Last 4 Games
Last 5 Games
Last 6 Games
Last 7 Games
Last 8 Games
Last 9 Games
Last 10 Games
Last 11 Games
Last 12 Games
Last 13 Games
Last 14 Games
Last 15 Games
All Months
January
February
March
April
May
June
July
August
September
October
November
December
Entire Season
Pre All-Star
Post All-Star
Advanced Filters
ROOKIE 2003-04
RECENT FILTERS GLOSSARY SHARE
67 Rows | Page
All
1
2
of 2
PLAYER TEAM AGE GP W L MIN OFFRTG DEFRTG NETRTG AST% AST/TO AST RATIO OREB% DREB% REB% TO RATIO EFG% TS% USG% PACE PIE
1
Luke Walton
LAL 24 72 50 22 10.0 101.4 100.0 1.4

In [121]:
# need to find some way of checking pages
stringified = str(soup)
listified = stringified.split('\n')
cut_list = listified[listified.index('of 2')+1:listified.index('PLAYER')]


['PLAYER TEAM AGE GP W L MIN OFFRTG DEFRTG NETRTG AST% AST/TO AST RATIO OREB% DREB% REB% TO RATIO EFG% TS% USG% PACE PIE',
 '1',
 'Luke Walton',
 'LAL 24 72 50 22 10.0 101.4 100.0 1.4 24.6 2.57 34.3 6.1 15.0 10.4 13.4 46.7 50.5 13.9 92.46 9.9',
 '2',
 'Darius Songaila',
 'SAC 26 73 49 24 13.3 107.9 100.9 7.0 7.7 1.12 11.9 10.7 15.6 13.2 10.7 48.7 54.1 16.4 94.47 8.9',
 '3',
 'Carmelo Anthony',
 'DEN 20 82 43 39 36.5 102.2 101.6 0.6 13.4 0.92 10.5 6.9 12.2 9.5 11.4 44.9 50.9 28.3 96.86 10.5',
 '4',
 'Raul Lopez',
 'UTA 24 82 42 40 19.8 95.4 97.6 -2.2 36.4 1.75 29.1 1.9 10.0 6.0 16.6 45.6 50.3 21.5 89.75 10.2',
 '5',
 'Josh Howard',
 'DAL 24 67 40 27 23.7 113.0 104.7 8.2 8.5 1.45 12.8 10.1 15.7 12.8 8.9 44.9 48.5 18.1 95.65 8.2',
 '6',
 'Sasha Pavlovic',
 'UTA 20 79 39 40 14.4 95.5 100.5 -4.9 9.8 0.90 11.1 4.8 12.9 8.8 12.4 42.2 46.3 19.8 89.21 4.6',
 '6',
 'Udonis Haslem',
 'MIA 24 75 39 36 24.0 98.7 98.8 0.0 4.9 0.69 7.8 12.3 18.8 15.5 11.3 45.9 52.1 15.7 90.74 9.5',
 '8',
 'Francisco 

In [294]:
rookies = pd.DataFrame(columns=cut_list[0].split(' '))
data_list = cut_list[1:]

# Select every third item (the names)
list_of_player_names = data_list[1::3]
rookies['PLAYER'] = pd.Series(list_of_player_names)

# Every third item here is the actual stats
rookies = pd.DataFrame([row.split(' ') for row in data_list[2::3]])

column_list = cut_list[0].replace('AST RATIO', 'AST_RATIO').replace('TO RATIO', 'TO_RATIO').split(' ')

# Set column names
rookies.columns = column_list[1:]

# add back in PLAYER name
rookies['PLAYER'] = pd.Series(list_of_player_names)

# reorder to put PLAYER first
rookies = rookies[column_list]

rookies['YEAR'] = 2003
#column_list = ['YEAR'].append(column_list)
new_column_list = column_list[0:2] + ['YEAR'] + column_list[2:]

rookies = rookies[new_column_list]

In [304]:
rookies

,PLAYER,TEAM,YEAR,AGE,GP,W,L,MIN,OFFRTG,DEFRTG,...,AST_RATIO,OREB%,DREB%,REB%,TO_RATIO,EFG%,TS%,USG%,PACE,PIE
0,Luke Walton,LAL,2003,24,72,50,22,10.0,101.4,100.0,...,34.3,6.1,15.0,10.4,13.4,46.7,50.5,13.9,92.46,9.9
1,Darius Songaila,SAC,2003,26,73,49,24,13.3,107.9,100.9,...,11.9,10.7,15.6,13.2,10.7,48.7,54.1,16.4,94.47,8.9
2,Carmelo Anthony,DEN,2003,20,82,43,39,36.5,102.2,101.6,...,10.5,6.9,12.2,9.5,11.4,44.9,50.9,28.3,96.86,10.5
3,Raul Lopez,UTA,2003,24,82,42,40,19.8,95.4,97.6,...,29.1,1.9,10.0,6.0,16.6,45.6,50.3,21.5,89.75,10.2
4,Josh Howard,DAL,2003,24,67,40,27,23.7,113.0,104.7,...,12.8,10.1,15.7,12.8,8.9,44.9,48.5,18.1,95.65,8.2
5,Sasha Pavlovic,UTA,2003,20,79,39,40,14.4,95.5,100.5,...,11.1,4.8,12.9,8.8,12.4,42.2,46.3,19.8,89.21,4.6
6,Udonis Haslem,MIA,2003,24,75,39,36,24.0,98.7,98.8,...,7.8,12.3,18.8,15.5,11.3,45.9,52.1,15.7,90.74,9.5
7,Francisco Elson,DEN,2003,28,62,36,26,14.1,95.4,96.6,...,11.2,8.3,17.8,13.1,12.2,47.2,49.8,13.0,94.78,7.1
8,Marcus Banks,BOS,2003,22,81,35,46,17.0,97.4,100.2,...,21.9,2.6,8.5,5.6,15.6,43.1,48.0,20.6,95.11,6.9
9,David West,NOH,2003,23,71,35,36,13.1,94.9,99.9,...,16.2,13.5,24.2,18.4,12.9,47.4,51.9,15.1,91.58,11.9
